In [42]:
from datasets import load_dataset

ds = load_dataset("UCSC-VLAA/Recap-DataComp-1B", split="preview",streaming=True)
ds

IterableDataset({
    features: ['url', 're_caption', 'org_caption', 'sha256', 'key', 're_clip_score', 'org_clip_score', 're_length', 'org_length', 're_gpt4v_score', 'org_gpt4v_score'],
    n_shards: 1
})

In [47]:
import itertools
limited_ds = itertools.islice(ds, 5)
limited_ds

In [48]:
for example in limited_ds:
    print(example)

{'url': 'https://images-na.ssl-images-amazon.com/images/I/41kzIaKFE+L._AC._SR180,230.jpg', 're_caption': 'A modern coffee machine with a digital display and two white coffee cups filled with coffee is shown. The machine has a stainless steel finish and is accompanied by a milk frothing pitcher with a white liquid inside. The coffee machine is placed on a surface with a white background.', 'org_caption': 'Saeco Xelsis Automatic Espresso Machine, SM7685/04, Stainless Steel', 'sha256': '501ac5e3066ff4cdad97dafd8916a65673b5e6d961c6e31e59dbeadd7fd7e3c7', 'key': '000000000008', 're_clip_score': 1.0, 'org_clip_score': 5.960464477539064e-08, 're_length': 50, 'org_length': 8, 're_gpt4v_score': 5, 'org_gpt4v_score': 5}
{'url': 'https://d2j6dbq0eux0bg.cloudfront.net/images/39097340/1998437688.jpg', 're_caption': 'A row of identical figures in black suits and ties is standing in a line against a white background.', 'org_caption': 'O 6-Car Flat End Offset Hopper Car Set, B&O 727022', 'sha256': '373

In [49]:
from datasets import load_dataset
import tensorflow as tf
import itertools
from tqdm import tqdm

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def create_example(sample):
    feature = {
        'image': _bytes_feature(sample['url'].encode('utf-8')),
        'url': _bytes_feature(sample['url'].encode('utf-8')),
        're_caption': _bytes_feature(sample['re_caption'].encode('utf-8')),
        'org_caption': _bytes_feature(sample['org_caption'].encode('utf-8')),
        'original_width': _int64_feature(sample.get('original_width', -1)),  # Use -1 as default if not present
        'original_height': _int64_feature(sample.get('original_height', -1))  # Use -1 as default if not present
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

def convert_to_tfrecord(dataset, output_file):
    with tf.io.TFRecordWriter(output_file) as writer:
        for sample in tqdm(dataset):
            tf_example = create_example(sample)
            writer.write(tf_example.SerializeToString())

# Load the dataset
ds = load_dataset("UCSC-VLAA/Recap-DataComp-1B", split="preview", streaming=True)

# Take the first 1000 samples
limited_ds = itertools.islice(ds, 1000)

# Convert the limited dataset to TFRecord
out_path = '~/recap_datacomp_1k.tfrecord'
convert_to_tfrecord(limited_ds, out_path)

2024-09-23 20:53:59.915021: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-23 20:53:59.937434: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-23 20:53:59.944199: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-23 20:53:59.959666: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-23 20:54:01.098659: W tensorflow/compiler/tf2